# Facial Feature/Ratio Generator

In [1]:
import cv2
import numpy as np
import dlib
from scipy.spatial import distance as dist
from tqdm import tqdm
import os
from statistics import stdev, mean


In [2]:

# dont need to run this

detector = dlib.get_frontal_face_detector()

# Load the predictor
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

img = cv2.cvtColor(cv2.imread("D:/image_data_copy/1/0/image_29.jpg"), cv2.COLOR_BGR2RGB)
faces = detector(img)
right_eye = []
left_eye = []
mouth = []
for face in faces:
    x1 = face.left() # left point
    y1 = face.top() # top point
    x2 = face.right() # right point
    y2 = face.bottom() # bottom point
   

 # Create landmark object
    landmarks = predictor(image=img, box=face)
    
    # Loop through all the points
    for n in range(42, 48):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        
        # Draw a circle
        cv2.circle(img=img, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)
        right_eye.append([x,y])
        
    for j in range(36, 42):
        x = landmarks.part(j).x
        y = landmarks.part(j).y
        
        # Draw a circle
        cv2.circle(img=img, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)
        left_eye.append([x,y])
        
    for i in range(48, 60):
        x = landmarks.part(i).x
        y = landmarks.part(i).y
        
        # Draw a circle
        cv2.circle(img=img, center=(x, y), radius=3, color=(0, 255, 0), thickness=-1)
        mouth.append([x,y])
    print(landmarks.part(42).x)        
print(right_eye)
print(left_eye)
print(mouth)
# show the image
cv2.imshow(winname="Face", mat=img)

# Delay between every fram
cv2.waitKey(delay=0)

# Close all windows
cv2.destroyAllWindows()


258
[[258, 139], [280, 120], [305, 121], [324, 139], [306, 146], [281, 144]]
[[87, 138], [108, 121], [134, 120], [161, 137], [135, 144], [109, 145]]
[[153, 329], [186, 305], [214, 293], [235, 297], [254, 293], [276, 304], [294, 326], [276, 338], [257, 344], [237, 346], [215, 344], [186, 339]]


In [2]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])
    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)
    # return the eye aspect ratio
    return ear

def avg_eye_aspect_ratio(left,right):
    if left == [] and right == []:
        return 0
    elif left == []:
        return eye_aspect_ratio(right)
    elif right == []:
        return eye_aspect_ratio(left)
    else:
        return ((eye_aspect_ratio(left) + eye_aspect_ratio(right))/2)

def mouth_aspect_ratio(mouth):
    if mouth == []:
        return 0
    A = dist.euclidean(mouth[2], mouth[10])
    B = dist.euclidean(mouth[3], mouth[9])
    C = dist.euclidean(mouth[4], mouth[8])
    D = dist.euclidean(mouth[0], mouth[6])
    mar= (A + B + C) / (3.0 * D)
    return mar

def mouth_over_eye(ear, mar):
    return mar/ear

# (ratio - mean(ratios))/standard deviation (ratios)

def normalization(target, data):
    return (target - mean(data))/stdev(data)


In [5]:
stdev([0.30,0.31,0.30])

0.005773502691896262

In [28]:
print("Left: " + str(eye_aspect_ratio(left_eye)))
print("Right: " + str(eye_aspect_ratio(right_eye)))
print("Avg: " + str(avg_eye_aspect_ratio(left_eye,right_eye)))
print("Mouth: " + str(mouth_aspect_ratio(mouth)))

Left: 0.3006659262401662
Right: 0.3138608051535341
Avg: 0.30726336569685014
Mouth: 0.3822614674525305


In [5]:
def create_train_data():
    path = "D:/training_data"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    training_data_RATIO = []
    for person in tqdm(os.listdir(path)):
        for state in tqdm(os.listdir(path + "/" + person)):
            if state == "0":
                eye_data = []
                mouth_data = []
                moe_data = []
                count = 0
                for img in os.listdir(path + "/" + person + "/" + state):
                    if count > 20:
                        break
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    right_eye = []
                    left_eye = []
                    mouth = []
            
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                     # Create landmark object
                        landmarks = predictor(image=img, box=face)
                        #if face landmarks can be detected
                        if landmarks.part(42).x != '':
                
                    # Loop through all the points
                            for n in range(42, 48):
                                x = landmarks.part(n).x
                                y = landmarks.part(n).y
        
                                right_eye.append([x,y])

                            for j in range(36, 42):
                                x = landmarks.part(j).x
                                y = landmarks.part(j).y
        
                                left_eye.append([x,y])
        
                            for i in range(48, 60):
                                x = landmarks.part(i).x
                                y = landmarks.part(i).y
        
                                mouth.append([x,y])
                            
                            ear = avg_eye_aspect_ratio(left_eye,right_eye)
                            mar = mouth_aspect_ratio(mouth)
                            moe = mouth_over_eye(ear, mar)
                            eye_data.append(ear)
                            mouth_data.append(mar)
                            moe_data.append(moe)
                    
                        break
                    count += 1
            counter = 0
            filter = 5
            for img in os.listdir(path + "/" + person + "/" + state):
                if counter % filter == 0:
                    
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    right_eye = []
                    left_eye = []
                    mouth = []
                    info = []
            
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                 # Create landmark object
                        landmarks = predictor(image=img, box=face)
                    #if face landmarks can be detected
                        if landmarks.part(42).x != '':
                
                # Loop through all the points
                            for n in range(42, 48):
                                x = landmarks.part(n).x
                                y = landmarks.part(n).y
        
                                right_eye.append([x,y])

                            for j in range(36, 42):
                                x = landmarks.part(j).x
                                y = landmarks.part(j).y
        
                                left_eye.append([x,y])
        
                            for i in range(48, 60):
                                x = landmarks.part(i).x
                                y = landmarks.part(i).y
        
                                mouth.append([x,y])
                        
                            ear = avg_eye_aspect_ratio(left_eye,right_eye)
                            mar = mouth_aspect_ratio(mouth)
                            moe = mouth_over_eye(ear, mar)
                            info.append(ear)
                            info.append(mar)
                            info.append(moe)
                            info.append(normalization(ear, eye_data))
                            info.append(normalization(mar, mouth_data))
                            info.append(normalization(moe, moe_data))
                            training_data_RATIO.append(np.array(info))
                        break
                counter += 1
                
                
    np.save("training_data_RATIO_shrunk", training_data_RATIO)
    print ("Done!")
    
def create_test_data():
    path = "D:/test_data"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    training_data_RATIO = []
    for person in tqdm(os.listdir(path)):
        for state in tqdm(os.listdir(path + "/" + person)):
            if state == "0":
                eye_data = []
                mouth_data = []
                moe_data = []
                count = 0
                for img in os.listdir(path + "/" + person + "/" + state):
                    if count > 20:
                        break
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    right_eye = []
                    left_eye = []
                    mouth = []
            
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                     # Create landmark object
                        landmarks = predictor(image=img, box=face)
                        #if face landmarks can be detected
                        if landmarks.part(42).x != '':
                
                    # Loop through all the points
                            for n in range(42, 48):
                                x = landmarks.part(n).x
                                y = landmarks.part(n).y
        
                                right_eye.append([x,y])

                            for j in range(36, 42):
                                x = landmarks.part(j).x
                                y = landmarks.part(j).y
        
                                left_eye.append([x,y])
        
                            for i in range(48, 60):
                                x = landmarks.part(i).x
                                y = landmarks.part(i).y
        
                                mouth.append([x,y])
                            
                            ear = avg_eye_aspect_ratio(left_eye,right_eye)
                            mar = mouth_aspect_ratio(mouth)
                            moe = mouth_over_eye(ear, mar)
                            eye_data.append(ear)
                            mouth_data.append(mar)
                            moe_data.append(moe)
                    
                        break
                    count += 1
            counter = 0
            filter = 5
            for img in os.listdir(path + "/" + person + "/" + state):
                if counter % filter == 0:
                    
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    right_eye = []
                    left_eye = []
                    mouth = []
                    info = []
            
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                 # Create landmark object
                        landmarks = predictor(image=img, box=face)
                    #if face landmarks can be detected
                        if landmarks.part(42).x != '':
                
                # Loop through all the points
                            for n in range(42, 48):
                                x = landmarks.part(n).x
                                y = landmarks.part(n).y
        
                                right_eye.append([x,y])

                            for j in range(36, 42):
                                x = landmarks.part(j).x
                                y = landmarks.part(j).y
        
                                left_eye.append([x,y])
        
                            for i in range(48, 60):
                                x = landmarks.part(i).x
                                y = landmarks.part(i).y
        
                                mouth.append([x,y])
                        
                            ear = avg_eye_aspect_ratio(left_eye,right_eye)
                            mar = mouth_aspect_ratio(mouth)
                            moe = mouth_over_eye(ear, mar)
                            info.append(ear)
                            info.append(mar)
                            info.append(moe)
                            info.append(normalization(ear, eye_data))
                            info.append(normalization(mar, mouth_data))
                            info.append(normalization(moe, moe_data))
                            training_data_RATIO.append(np.array(info))
                        break
                counter += 1
                
                
    np.save("test_data_RATIO_shrunk", training_data_RATIO)
    print ("Done!")

In [6]:
create_train_data()
create_test_data()

100%|██████████| 48/48 [07:17<00:00,  9.11s/it]


Done!


100%|██████████| 12/12 [01:35<00:00,  7.97s/it]

Done!


In [31]:
test[0]

array([ 0.3651145 ,  0.39587182,  1.08424019,  0.06209334, -0.55851786,
       -0.34768605])

# Extras

In [7]:
def create_test_data():
    path = "D:/first 1 mintue"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
   
    for person in tqdm(os.listdir(path)):
        for state in tqdm(os.listdir(path + "/" + person)):
            test_data_RATIO = []
            if state == "0":
                eye_data = []
                mouth_data = []
                moe_data = []
                count = 0
                for img in os.listdir(path + "/" + person + "/" + state):
                    if count > 20:
                        break
                    img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                    faces = detector(img)
                    right_eye = []
                    left_eye = []
                    mouth = []
            
                    for face in faces:
                        x1 = face.left() # left point
                        y1 = face.top() # top point
                        x2 = face.right() # right point
                        y2 = face.bottom() # bottom point
   

                     # Create landmark object
                        landmarks = predictor(image=img, box=face)
                        #if face landmarks can be detected
                        if landmarks.part(42).x != '':
                
                    # Loop through all the points
                            for n in range(42, 48):
                                x = landmarks.part(n).x
                                y = landmarks.part(n).y
        
                                right_eye.append([x,y])

                            for j in range(36, 42):
                                x = landmarks.part(j).x
                                y = landmarks.part(j).y
        
                                left_eye.append([x,y])
        
                            for i in range(48, 60):
                                x = landmarks.part(i).x
                                y = landmarks.part(i).y
        
                                mouth.append([x,y])
                            
                            ear = avg_eye_aspect_ratio(left_eye,right_eye)
                            mar = mouth_aspect_ratio(mouth)
                            moe = mouth_over_eye(ear, mar)
                            eye_data.append(ear)
                            mouth_data.append(mar)
                            moe_data.append(moe)
                    
                        break
                    count += 1
                
            for img in os.listdir(path + "/" + person + "/" + state):
                img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                faces = detector(img)
                right_eye = []
                left_eye = []
                mouth = []
                info = []
            
                for face in faces:
                    x1 = face.left() # left point
                    y1 = face.top() # top point
                    x2 = face.right() # right point
                    y2 = face.bottom() # bottom point
   

                 # Create landmark object
                    landmarks = predictor(image=img, box=face)
                    #if face landmarks can be detected
                    if landmarks.part(42).x != '':
                
                # Loop through all the points
                        for n in range(42, 48):
                            x = landmarks.part(n).x
                            y = landmarks.part(n).y
        
                            right_eye.append([x,y])

                        for j in range(36, 42):
                            x = landmarks.part(j).x
                            y = landmarks.part(j).y
        
                            left_eye.append([x,y])
        
                        for i in range(48, 60):
                            x = landmarks.part(i).x
                            y = landmarks.part(i).y
        
                            mouth.append([x,y])
                        
                        ear = avg_eye_aspect_ratio(left_eye,right_eye)
                        mar = mouth_aspect_ratio(mouth)
                        moe = mouth_over_eye(ear, mar)
                        info.append(ear)
                        info.append(mar)
                        info.append(moe)
                        info.append(normalization(ear, eye_data))
                        info.append(normalization(mar, mouth_data))
                        info.append(normalization(moe, moe_data))
                        test_data_RATIO.append(np.array(info))
                    break
                
                
            np.save("./minute/" + person + "/" + state + "/" + "test_RATIO", test_data_RATIO)
    print ("Done!")

In [8]:
create_test_data()

100%|██████████| 11/11 [21:16<00:00, 116.04s/it]

Done!


In [33]:
test = np.load("test_data_RATIO_v4.npy")
test.shape

(21200, 6)

# non normalized data

In [15]:
def create_test_data():
    path = "D:/test_data"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    test_data_RATIO = []
    for person in tqdm(os.listdir(path)):
        for state in os.listdir(path + "/" + person):
            for img in os.listdir(path + "/" + person + "/" + state):
                img = cv2.cvtColor(cv2.imread(path + "/" + person + "/" + state + "/" + img), cv2.COLOR_BGR2RGB)
                faces = detector(img)
                right_eye = []
                left_eye = []
                mouth = []
                info = []
            
                for face in faces:
                    x1 = face.left() # left point
                    y1 = face.top() # top point
                    x2 = face.right() # right point
                    y2 = face.bottom() # bottom point
   

                 # Create landmark object
                    landmarks = predictor(image=img, box=face)
            
                    #check if face landmarks can be detected
                    if landmarks.part(42).x !='':
    
                # Loop through all the points
                        for n in range(42, 48):
                            x = float(landmarks.part(n).x)
                            y = float(landmarks.part(n).y)
        
                            right_eye.append([x,y])

                        for j in range(36, 42):
                            x = float(landmarks.part(j).x)
                            y = float(landmarks.part(j).y)
        
                            left_eye.append([x,y])
        
                        for i in range(48, 60):
                            x = float(landmarks.part(i).x)
                            y = float(landmarks.part(i).y)
        
                            mouth.append([x,y])
            
                        info.append(avg_eye_aspect_ratio(left_eye,right_eye))
                        info.append(mouth_aspect_ratio(mouth))
                        test_data_RATIO.append(np.array(info))    
                    break
                
    np.save("test_data_RATIO_v3", test_data_RATIO)
    print ("Done!")

In [16]:
create_test_data()

100%|██████████| 12/12 [07:10<00:00, 35.85s/it]

Done!


In [4]:
def create_test_data_video():
    path = "D:/test_data/50/0"
    detector = dlib.get_frontal_face_detector()
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    test_data_RATIO = []
    #for person in tqdm(os.listdir(path)):
        #for state in os.listdir(path + "/" + person):
    for img in tqdm(os.listdir(path)):
        #if counter < 0:
        #check if face is recognizable
        img = cv2.cvtColor(cv2.imread(path + "/" + img), cv2.COLOR_BGR2RGB)
        faces = detector(img)
        right_eye = []
        left_eye = []
        mouth = []
        info = []
            
        for face in faces:
            x1 = face.left() # left point
            y1 = face.top() # top point
            x2 = face.right() # right point
            y2 = face.bottom() # bottom point
   

            # Create landmark object
            landmarks = predictor(image=img, box=face)
            
            #check if face landmarks can be detected
            if landmarks.part(42).x !='':
    
        # Loop through all the points
                for n in range(42, 48):
                    x = float(landmarks.part(n).x)
                    y = float(landmarks.part(n).y)
        
                    right_eye.append([x,y])

                for j in range(36, 42):
                    x = float(landmarks.part(j).x)
                    y = float(landmarks.part(j).y)
        
                    left_eye.append([x,y])
        
                for i in range(48, 60):
                    x = float(landmarks.part(i).x)
                    y = float(landmarks.part(i).y)
        
                    mouth.append([x,y])
            
                info.append(avg_eye_aspect_ratio(left_eye,right_eye))
                info.append(mouth_aspect_ratio(mouth))
                test_data_RATIO.append(np.array(info))    
            break
                
    np.save("ratio_50_0", test_data_RATIO)
    print ("Done!")
    
create_test_data_video()

100%|████████████████████████████████████████████████████████████████████████████████| 606/606 [00:12<00:00, 47.25it/s]


Done!
